In [12]:
# coding: utf-8

import pandas as pd
import numpy as np
from matplotlib import rcParams
import matplotlib.pyplot as plt
from collections import defaultdict
from datetime import datetime
import matplotlib.patches as mpatches
import matplotlib
import time
import math
from operator import itemgetter
from scipy.spatial import distance
%matplotlib inline  

rcParams['font.family'] = 'NanumGothic'
rcParams.update({'font.size': 12})
matplotlib.style.use('ggplot')
pd.options.display.max_rows=14


## 무비 렌즈 데이터로 별점을 예측해 보자 
* User Based 별점 예측 
* Item(Movie) Based 별점 예측

Movie Lens 데이터 로드 
http://grouplens.org/datasets/movielens/

In [167]:
class ub_ressys:
    home_dir = '/Users/goodvc/Documents/data-analytics/movie-recommendation/'
    nn_func = None
    sim_func = None
    
    
    def __init__(self, dataname='ml-latest-small'):
        self.ds_type = dataname
        self.nn_func = self.nearest_neighbor_user
        self.sim_func = self.distance_euclidean
        self.movieLensDataLoad()
        self.UM_matrix_ds = self.ratings.pivot(index='userId', columns='movieId', values='rating')
    
    def movieLensDataLoad(self):
        ## user 영화 별점 data 
        self.ratings = pd.read_csv('{home}/{type}/{name}'.format( home=self.home_dir, type=self.ds_type, name='ratings.csv'))

        ## movie meta(타이트,장르) data 
        self.movies = pd.read_csv('{home}/{type}/{name}'.format( home=self.home_dir, type=self.ds_type, name='movies.csv'))

        ## user가 영화에 tag를 기입한 data
        self.tags = pd.read_csv('{home}/{type}/{name}'.format( home=self.home_dir, type=self.ds_type, name='tags.csv'))


    def compute(self):
        ## 1. U X M vector Matrix를 만든다. 
        self.UM_matrix_ds = self.ratings.pivot(index='userId', columns='movieId', values='rating')
        
        ## pre-clustering
        #self.graph_cut()

    ## filtering movies by rating frequency
    def cut_movie(self, threshold=5):
        movie_rating_count = self.ratings.groupby(['movieId'])['rating'].count()
        except_list = movie_rating_count[movie_rating_count<threshold].index
        self.ratings = self.ratings[(~self.ratings['movieId'].isin(except_list))]
        return (self.ratings.count()[0])
        
    ## distance functions
    def distance_cosine(self, a, b):        return 1-distance.cosine(a,b)
    def disance_corr(self, a, b):           return 1-distance.correlation (a,b)
    def distance_euclidean(self, a, b):     return 1/(distance.euclidean(a,b)+1)
    
    ## none-cluster nn functions
    def nearest_neighbor_user(self, user, topN, simFunc ) :  
        u1 = self.UM_matrix_ds.loc[user].dropna()
        ratedIndex = u1.index
        nn = {}

        ## Brote Force Compute
        for uid, row in self.UM_matrix_ds.iterrows():
            interSectionU1 = []
            interSectionU2 = []
            if uid==user:
                continue
            for i in ratedIndex:
                if False==math.isnan(row[i]):
                    interSectionU1.append(u1[i])
                    interSectionU2.append(row[i])
            interSectionLen = len(interSectionU1)

            ## At least 3 intersection items
            if interSectionLen < 3 :
                continue

            ## similarity functon
            sim = simFunc(interSectionU1,interSectionU2)

            if  math.isnan(sim) == False:
                nn[uid] = sim

        ## top N returned         
        return sorted(nn.items(),key=itemgetter(1))[:-(topN+1):-1]

    ## rating prediction by user
    def predictRating(self, userid, nn=100) :
        ## neighboorhood 
        neighbor = self.nn_func(userid, nn, self.sim_func)
        neighbor_id = [id for id,sim in neighbor]

        ## neighboorhood's movie : al least 4 ratings
        neighbor_movie = self.UM_matrix_ds.loc[neighbor_id]\
                        .dropna(1, how='all', thresh = 4 )
        neighbor_dic = (dict(neighbor))
        ret = [] # ['movieId', 'predictedRate']

        ## rating predict by my similarities 
        for movieId, row in neighbor_movie.iteritems():
            jsum, wsum = 0, 0
            for v in row.dropna().iteritems():
                sim = neighbor_dic.get(v[0],0)
                jsum += sim
                wsum += (v[1]*sim)
            ret.append([movieId, wsum/jsum])

        return ret

    
    ## evaluation 
    def evaluation(self, user_list, acc_func, nn=50):
        eval_ds = pd.merge(self.ratings, 
                           self.ratings[['movieId','rating']].groupby(['movieId']).mean().reset_index(), 
                           on='movieId', how='left')
        eval_ds = eval_ds.rename(columns= {'rating_x':'rating', 'rating_y':'mean_rating'})
        st = time.time()
        name = self.sim_func.__name__
        eval_ds[name] = 0
        for userId in user_list:
            for x in self.predictRating(userId, nn):
                eval_ds.loc[(eval_ds.userId==userId) & (eval_ds.movieId==x[0]),name]=x[1]
        print('elapsed', round(time.time()-st,2), 'sec')
        eval_ds = eval_ds[eval_ds[name]>0]
        
        pzn_acc = acc_func(eval_ds, 'rating', name)
        nopzn_acc = acc_func(eval_ds, 'rating', 'mean_rating')
        return (pzn_acc, nopzn_acc, eval_ds)
    
    def graph_cut(self, threshold=3 ):
        st = time.time()
        grouped = self.ratings.groupby('movieId')
        userlink = defaultdict(int)
        for name, g in grouped:
            users = g['userId'].values
            cnt = users.size
            if cnt<2:
                continue

            for n in range(0,cnt):
                for m in range(n+1,cnt):
                    key = "{0}:{1}" if users[n]<users[m] else "{1}:{0}"
                    key = key.format(users[n], users[m])
                    userlink[key] = userlink[key] + 1

        self.neighbors = defaultdict(list)
        n=0
        for k, v in userlink.items():
            if (v) > threshold:
                nm = k.split(':')
                if len(nm) == 2:
                    self.neighbors[int(nm[0])].append(int(nm[1]))
                    self.neighbors[int(nm[1])].append(int(nm[0]))
                    n = n+1
        cnt = len(self.UM_matrix_ds.index)
        return (time.time()-st, n/(cnt**2-cnt))

    def clustered_nearest_neighbor_user(self, user, topN, simFunc ) :  
        u1 = self.UM_matrix_ds.loc[user].dropna()
        ratedIndex = u1.index
        nn = {}

        members = self.neighbors[user]
        ## pre-clustered Compute
        for uid in members:
            interSectionU1 = []
            interSectionU2 = []
            if uid==user:
                continue
            row = self.UM_matrix_ds.loc[uid]
            for i in ratedIndex:
                if False==math.isnan(row[i]):
                    interSectionU1.append(u1[i])
                    interSectionU2.append(row[i])
            interSectionLen = len(interSectionU1)


        ## At least 3 intersection items
            if interSectionLen < 3 :
                continue

            ## similarity functon
            sim = simFunc(interSectionU1,interSectionU2)

            if  math.isnan(sim) == False:
                nn[uid] = sim

        ## top N returned         
        return sorted(nn.items(),key=itemgetter(1))[:-(topN+1):-1]

    ## accuracy measure functions
    def RMSE(self, X, left_col, right_col):
        return(np.sqrt(np.mean((X[left_col] - X[right_col])**2)))
    def MAE(self, X, left_col, right_col):
        return(np.mean(np.absolute(X[left_col] - X[right_col])) )
ub = ub_ressys()

* 별점 예측 객체 생성
 * euclidean distance
 * none-clustered nearest neighborhood


In [190]:
ub = ub_ressys()

In [119]:
ub.nearest_neighbor_user(1,20,ub.distance_euclidean)

[(651, 0.5),
 (591, 0.5),
 (424, 0.5),
 (513, 0.4721359549995794),
 (473, 0.4721359549995794),
 (436, 0.4494897427831781),
 (97, 0.4494897427831781),
 (81, 0.4494897427831781),
 (57, 0.4494897427831781),
 (510, 0.4142135623730951),
 (312, 0.4142135623730951),
 (300, 0.4142135623730951),
 (240, 0.4142135623730951),
 (220, 0.4142135623730951),
 (93, 0.4142135623730951),
 (67, 0.4142135623730951),
 (16, 0.4142135623730951),
 (12, 0.4142135623730951),
 (696, 0.4),
 (288, 0.4)]

In [122]:
## 1번 유저 별점 예측
ub.predictRating(1, nn=100)

[[1, 3.4973757725576129],
 [2, 2.7747861526493329],
 [3, 3.1137190994529718],
 [5, 3.7161621943950065],
 [7, 3.6761266239129329],
 [10, 3.0140608684784622],
 [11, 3.8072629433412737],
 [16, 3.5236389990063142],
 [17, 4.0564496803920296],
 [19, 2.5008287819113257],
 [21, 3.649250062205434],
 [25, 3.4993114631374516],
 [34, 3.1591863510187821],
 [36, 4.5362050315446636],
 [39, 3.0572343242590638],
 [47, 4.1461922072466191],
 [50, 4.6291333840685196],
 [58, 3.8056727568416218],
 [62, 3.8266779938410362],
 [64, 2.7720015923782153],
 [95, 3.562572880882525],
 [104, 3.7246411662957439],
 [110, 4.0446774455572809],
 [111, 3.8174805450628155],
 [141, 3.8184761216903294],
 [150, 3.6393638353377944],
 [151, 4.5374647163504056],
 [153, 3.0550438924610943],
 [161, 3.9208845348800416],
 [165, 3.318787814709848],
 [168, 2.8431105992410308],
 [185, 3.1572235872383585],
 [186, 2.8243305690765879],
 [203, 4.2401978961145543],
 [208, 2.9833541068444007],
 [223, 3.3977992935652548],
 [231, 2.287905428509

In [123]:
## 1번 유저 별점 예측
len(ub.predictRating(1, nn=100))

207

In [ ]:
## 1번 유저 별점 예측
len(ub.predictRating(1, nn=100))

In [192]:
## nn 개수별 별점 예측 개수
st = time.time()
for n in range(10,100,10):
    print(n, len(ub.predictRating(3,nn=n)))
time.time()-st

10 2
20 36
30 80
40 128
50 184
60 246
70 311
80 359
90 443


5.955229043960571

In [193]:
ub.UM_matrix_ds.columns.size

8552

In [194]:
print('cut ratings', ub.cut_movie(5))
ub.compute()

cut ratings 90637


In [195]:
ub.UM_matrix_ds.columns.size

3313

In [196]:
## nn 개수별 별점 예측 개수
st = time.time()
for n in range(10,100,10):
    print(n, len(ub.predictRating(3,nn=n)))
time.time()-st

10 2
20 36
30 80
40 128
50 184
60 246
70 311
80 359
90 443


5.758426904678345

In [175]:
## pre-clustered nearest_neighbor_user
ub.nn_func = ub.clustered_nearest_neighbor_user
ub.graph_cut(3)

(9.016196966171265, 0.3066260824141603)

In [160]:
## nn 개수별 별점 예측 개수
ub.nn_func = ub.clustered_nearest_neighbor_user
st = time.time()
for u in range(1,10):
    st2=time.time()
    print(u, len(ub.predictRating(u,nn=50)), time.time()-st2)
print('taken', time.time()-st)

1 75 1.3762969970703125
2 218 0.3569459915161133
3 267 0.563269853591919
4 161 0.7298669815063477
5 914 0.26584601402282715
6 693 0.43865013122558594
7 252 0.8145508766174316
8 129 0.8499939441680908
9 176 0.7811188697814941
taken 6.180775165557861


In [161]:
## nn 개수별 별점 예측 개수
ub.nn_func = ub.nearest_neighbor_user
st = time.time()
for u in range(1,10):
    st2=time.time()
    print(u, len(ub.predictRating(u,nn=50)), time.time()-st2)
print('taken', time.time()-st)

1 79 1.4130451679229736
2 203 0.42317700386047363
3 184 0.6401231288909912
4 122 0.6389970779418945
5 630 0.3234999179840088
6 597 0.9553661346435547
7 172 0.8523831367492676
8 127 0.8880388736724854
9 128 0.8787670135498047
taken 7.0174241065979


In [ ]:
ub.UM_matrix_ds

In [103]:
#ub.nearest_neighbor_user(10,20, ub.distance_euclidean)
result = ub.evaluation(ub.UM_matrix_ds.index.values.tolist(),ub.RMSE, 50)

elapsed 1615.05 sec


In [116]:
ub.RMSE(result[2],'rating', 'distance_euclidean')

0.51365490423561666